In [ ]:
%run common.ipynb

# Trials

In [ ]:
def trial(nn='CNN', N_range=[5,6], random_states=[0, 17], hstack_zeros=False):
    print('Processing...')
    print()

    a_N = np.array([])
    a_sqrt_mse = np.array([])
    a_avg_abs_errors = np.array([])
    a_std_abs_errors = np.array([])

    # change N [32..2048]
    for i in N_range:

        N = 2**i

        a_N = np.append(a_N, N)

        # data generation (fix random state)
        DATA_RANDOM_STATE = 0
        fix_random(seed=DATA_RANDOM_STATE)
        X, y, M = generate_data(N=N, 
                                num_alphas=100, 
                                samples_per_alpha=100, 
                                random_alpha=True, 
                                max_M_only=False, 
                                hstack_zeros=hstack_zeros, 
                                random_state=DATA_RANDOM_STATE)

        # reshape X for Conv1D
        if nn.startswith('CNN'):
            X = X.reshape(X.shape[0], X.shape[1], 1)
        elif 'DNN_WIN' == nn:
            X = data_hstack_win_max(X, window_sizes=None)

        # split train/test
        print('input.shape:', X.shape)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=DATA_RANDOM_STATE)

        # create model
        if 'CNNM' == nn: # CNN-multi-layers
            model = create_cnn_model_multi_layer(X.shape[1], filters=32)
        elif nn.startswith('CNN'):
            model = create_cnn_model(X.shape[1], filters=32)
        elif nn.startswith('DNN'):
            model = create_dnn_model(X_train.shape[1], 
                            layers=[256,256], 
                            activation='relu', 
                            init=keras.initializers.HeUniform(seed=DATA_RANDOM_STATE), 
                            batch_normalization=True, 
                            dropout=0.0, 
                            optimizer=Adam(lr=1e-2), 
                            k_reg=True, 
                            k_reg_lr=1e-5, 
                            a_reg=True, 
                            a_reg_lr=1e-5)
        else:
            raise RuntimeError("nn: '{}' not supported".format(nn))

        # average sqrt_mse among different random states (fixed N)
        avg_sqrt_mse = 0

        # array of absolute errors
        avg_abs_errors = np.zeros_like(y_test)

        # change random state
        for rs in random_states:

            # fix random
            fix_random(seed=rs)
        
            # training
            model, history = train(model, X_train, y_train, batch_size=32, random_state=rs)

            # predict
            y_pred = model.predict(X_test).flatten()

            # mse
            mse = mean_squared_error(y_test, y_pred)

            # sqrt_mse
            sqrt_mse = np.sqrt(mse)

            # avg_sqrt_mse (accumulator)
            avg_sqrt_mse += sqrt_mse
            
            #print('N = {}, M = {}, random_state = {}    =>    sqrt_mse = {:.6f}'.format(N, M, rs, sqrt_mse))

            # absolute errors
            avg_abs_errors += np.abs(y_test - y_pred)

        # avg errors across random_states
        avg_abs_errors = avg_abs_errors / len(random_states)
        
        # avg errors across samples (t_test)
        a_avg_abs_errors = np.append(a_avg_abs_errors, np.average(avg_abs_errors))

        # std across samples
        a_std_abs_errors = np.append(a_std_abs_errors, np.std(avg_abs_errors))

        # avg_sqrt_mse
        avg_sqrt_mse = avg_sqrt_mse / len(random_states)
        a_sqrt_mse = np.append(a_sqrt_mse, avg_sqrt_mse)
        print('N: {} \t avg_sqrt_mse = {:.6f}'.format(N, avg_sqrt_mse))
        print()

    # return a_N, a_sqrt_mse, a_avg_abs_errors, a_std_abs_errors 
    return {
        'nn': nn, 
        'a_N':a_N, 
        'a_sqrt_mse':a_sqrt_mse, 
        'a_avg_abs_errors':a_avg_abs_errors, 
        'a_std_abs_errors':a_std_abs_errors
    }

# CONSTANTS

In [ ]:
# N_RANGE=[6,7,8,9]; RANDOM_STATES=[0, 17]
N_RANGE=range(5, 12); RANDOM_STATES=[0, 3, 5, 8, 11, 16, 17, 20, 21, 24]

trials_out = []

RUN_DNN = True
RUN_DNN_WIN = True
RUN_CNN = True
RUN_CNN_ZEROS_WIN = True
RUN_CNNM = False


# DNN trials

In [ ]:
if RUN_DNN:
    trials_out.append(trial(nn='DNN', N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES))

# CNN trials

In [ ]:
if RUN_CNN:
    trials_out.append(trial(nn='CNN', N_range=N_RANGE, random_states=RANDOM_STATES))

In [ ]:
if RUN_CNN_ZEROS_WIND:
    trials_out.append(trial(nn='CNN_ZEROS_WIN', N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES))

# CNN multi-layer trials

In [ ]:
# TODO
if RUN_CNNM:
    trials_out.append(trial(nn='CNNM', N_range=N_RANGE, random_states=RANDOM_STATES))

# DNN trials with data augmentation (hstack windows sums of sizes [2,4,8,...])

In [ ]:
if RUN_DNN_WIN:
    trials_out.append(trial(nn='DNN_WIN', N_range=N_RANGE, random_states=RANDOM_STATES))

# plot log(N) vs sqrt_mse

In [ ]:
def plot_sqrt_mse():
    
    plt.figure(figsize=(15, 4))
    
    ax1 = plt.subplot(1, 2, 1)

    ax1.set(title='sqrt_mse', xlabel='log(N)', ylabel='sqrt_mse')

    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, out in enumerate(trials_out):
        x = np.log10(out['a_N'])
        y = out['a_sqrt_mse']
        std = out['a_std_abs_errors']
        ax1.plot(x, y, linewidth=2, color=colors[i], marker='o', markersize=6, label=out['nn'])
        ax1.fill_between(x, y - std, y + std, facecolor=colors[i], alpha=0.3)
    
    plt.legend(loc='upper right')

plot_sqrt_mse()


# print and save statistics

In [ ]:
dict_list = []
for t in trials_out:
    dict_list.append(('N', t['a_N']))
for t in trials_out:
    dict_list.append(('sqrt_mse_{}'.format(t['nn']), t['a_sqrt_mse']))
for t in trials_out:
    dict_list.append(('std_{}'.format(t['nn']), t['a_std_abs_errors']))
for t in trials_out:
    dict_list.append(('avg_abs_error_{}'.format(t['nn']), t['a_avg_abs_errors']))

df = pd.DataFrame(dict(dict_list))

df = df.set_index('N')

columns=[]
for t in trials_out:
    columns.append(('sqrt_MSE',t['nn']))
for t in trials_out:
    columns.append(('STD',t['nn']))
for t in trials_out:
    columns.append(('avg_abs_err',t['nn']))

df.columns=pd.MultiIndex.from_tuples(columns)

df.to_csv('yulesimon.csv')

print('err = y_test - y_pred')
df